In [1]:
import os
import torch
import torch.nn as nn 
import time
from tqdm import tqdm
from copy import copy

from torch_geometric.utils import subgraph
from torch_geometric.nn import Node2Vec
from torch.utils.data import DataLoader
from torch_geometric.datasets import Planetoid, NELL, Twitch, CitationFull
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.utils import structured_negative_sampling



/home/aemad/.envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def evaluate(x, edge_index, labels):
    from sklearn.metrics import roc_auc_score, average_precision_score
    s_ = []; t_= []; l= []
    for i, e in enumerate(edge_index):
        u = e[0]
        v = e[1]
        
        if u < x.shape[0] and v < x.shape[0]:
            s_.append(u)
            t_.append(v)
            l.append(labels[i])
            
    s = torch.tensor(s_)
    t= torch.tensor(t_)
    labels = torch.tensor(l)
    s_emb = x[s].detach()
    t_emb = x[t].detach()

    scores = s_emb.mul(t_emb).sum(dim=-1).cpu().numpy()
    auc = roc_auc_score(y_true=labels, y_score=scores)
    ap = average_precision_score(y_true=labels, y_score=scores)
    return auc, ap

In [4]:
import pickle    
def save_file(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)
    /home/aemad/PycharmProjects/project_slkd/
def open_file(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data
# 
name = 'CS'
inductive = False

train_data = open_file('/home/aemad/PycharmProjects/project_slkd/datasplits/'+name+'_train_data.pickle')
valid_data = open_file('/home/aemad/PycharmProjects/project_slkd/datasplits/'+name+'_valid_data.pickle')
test_data = open_file('/home/aemad/PycharmProjects/project_slkd/datasplits/'+name+'_test_data.pickle')


In [5]:
device = torch.device('cuda:0')
model = Node2Vec(train_data.edge_index, embedding_dim=128, walk_length=20,
                 context_size=5, walks_per_node=40,
                 num_negative_samples=1, p=1, q=1, sparse=True).to(device)

In [6]:
loader = model.loader(batch_size=1000, shuffle=True, num_workers=4)  # data loader to speed the train 
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)  # initzialize the optimizer 


In [7]:
def train():
    model.train()  # put model in train model
    total_loss = 0
    for pos_rw, neg_rw in tqdm(loader):
        optimizer.zero_grad()  # set the gradients to 0
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))  # compute the loss for the batch
        loss.backward()
        optimizer.step()  # optimize the parameters
        total_loss += loss.item()
    return total_loss / len(loader)


In [8]:
best_model = None
best = 0
for epoch in range(1, 1000):
    #torch.manual_seed(10)
    loss = train()
    auc, ap = evaluate(model(torch.arange(torch.max(train_data.edge_index), device=device)),
                      valid_data.edge_label_index.T,
                      valid_data.edge_label.long())
    if ap> best:
        best = ap 
        best_model = model
        
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}'
             f"validation AUC: {auc}, AP: {ap}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.30it/s]


Epoch: 10, Loss: 1.3089validation AUC: 0.9407452973818329, AP: 0.9262902963247577


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.29it/s]


Epoch: 20, Loss: 0.8669validation AUC: 0.9887313889607534, AP: 0.9889526086742733


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.27it/s]


...

Epoch: 370, Loss: 0.7827validation AUC: 0.994211704629191, AP: 0.9955377669449172


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  6.28it/s]


Epoch: 380, Loss: 0.7827validation AUC: 0.9940810639177592, AP: 0.9954783445982947


 16%|███████████████████▎                                                                                                      | 3/19 [00:00<00:04,  3.25it/s]


KeyboardInterrupt: 

In [9]:
from copy import deepcopy

In [10]:
auc, ap = evaluate(best_model(torch.arange(torch.max(train_data.edge_index), device=device)),
                  test_data.edge_label_index.T,
                  test_data.edge_label.long())
print(f"validation AUC: {auc}, AP: {ap}")


validation AUC: 0.9926616875295136, AP: 0.994756476712245


In [ ]:
save_file(best_model, name+"_dw.pickle")